# 2. DVC

[Data Version Control](https://dvc.org) is a Data Versioning tool that uses lightweight pointers that indicate where your data is stored (S3 buckets, Google Drive, GCS for example). Additionally, it stores information about the experiments you run using the mentioned data. On top of that, it allows you to make or retrieve changes in or from a remote server. And the best thing is that you will use commands very similar to the commands in Git!

# Installation

There are many ways to install DVC in your computer, and it works very well across OSs. One solution that will solve for Windows, Mac, and Linux is running:
```
conda install -c conda-forge mamba
mamba install -c conda-forge dvc
```

Then, you might want to install the necessary package corresponding to the remote storage service you are going to use. You example, if you plan to use S3 storage, you can run the following:

```
mamba install -c conda-forge dvc-s3
```
Or if you are going to use google-drive
```
mamba install -c conda-forge dvc-gdrive
```
This will install the corresponding 

# Initializing DVC

DVC works very well along with Git projects, and thus, we will create a Git repository where we will initialize DVC. Thus, create a new repository (anywhere in your local machine).

```
mkdir test_dvc && cd test_dvc
git init
```
And once inside, you initialize dvc:
```
dvc init
```

Just by doing so, there are many new files in your initially empty repo. 
<div style="text-align:center"><img src="images/test_dvc.png" width=175/></div>

If you run `git status` you will see that all created files are already staged, and it would be a good idea to commit them. So you could directly run:
```
git commit -m "DVC Init"
```

# DVC for Data Versioning

One of the main ideas of DVC is to harness all the power of Git and use that for large datasets, so next time you clone a repo, you can obtain a really extensive dataset with the corresponding model that was trained using that data.

In this section we will see how to store local data remotely using commands similar to Git. For the example, you can download a `data.xml`, a dataset provided by the DVC team to experiment with.
```
dvc get https://github.com/iterative/dataset-registry get-started/data.xml -o data/data.xml
```

The command above is downloading an `xml` file, and you are already using dvc commands! In this case, you are using `get` to download data tracked by DVC or Git, think about it as `wget`. You can check more about that command in [this link here](https://dvc.org/doc/start/data-and-model-versioning)



Now, you can put this file into stage state the same way you would do it using Git
```
dvc add data/data.xml
```

After running that, you will see that a `.gitignore` file and a `data.xml.dvc` file have been created. `.gitignore` will prevent you from uploading the whole dataset to Git, and `data.xml.dvc` will store information about the 'pointer' to find the data once we push it. Don't forget to add them to the stage state:

```
git add 'data\data.xml.dvc' 'data\.gitignore'
git commit -m 'Add DVC tracker'
```

Talking about push, it is time to push our data. For this example, we are going to use an Amazon S3 bucket, which is highly recommended due to its simplicity. 

Before that make sure you have already ran:

```
mamba install -c conda-forge dvc-s3
```

That way, DVC will know your credentials when you try to push something. On that note, the second thing you have to provide are your credentials. If you already have an S3 bucket, you have probably set up your aws configuration data. Otherwise, you can create an IAM user and create a user with administration permissions. For more information, go to the Cloud Basics module in the Data Engineering unit.

Go to the bucket you want to upload your data to, and create a new folder. In my case, I will create a folder named AiCore_DVC

<div style="text-align:center"><img src="images/DVC_S3.png" width=600/></div>

Once created, you have to copy the URL of the S3 folder, so you add a remote bucket to DVC (just like when you add a remote url to git). Next time you make a push, the data tracked by DVC will be uploaded to this folder.

<div style="text-align:center"><img src="images/DVC_S3_2.png" width=600/></div>

To set the remote DVC storage, you have to execute the following command:
```
dvc remote add -d <url_name> s3://<bucket_name>/<folder_name>/
```
In this case, <url_name> makes reference to the name you want to save that URL as (the same thing you used `origin` for in GitHub). In this case, we are going to simply call it storage. So in the example, you would run:
```
dvc remote add -d storage s3://dvc-aicore/AiCore_DVC/
```

_If you want to overwrite an existing remote URL, you can add -f, for force, after -d_

Before proceding make sure you commit the new changes. The configurations about the remote storage will be kept in the created file `.dvc/config`. Thus, run:
```
git add .dvc/config 
git commit -m "Configures remote data storage"
```
And once commited, it is actually time to push the data. Everything is set up, so you don't have to set anything else, simply push the data running:
```
dvc push
```
And you will see that there is a surprise in your bucket! _It looks like it is something random, but this is the data you have just pushed_

Pulling changes is done by, as you expect, running:
```
dvc pull
```
Right now, it would be pointless running it, but anyone in your team can retrieve the data you have just uploaded after cloning the repo.

If you make a change to your data, you can also update the remote data. Let's say that you have additional data, in this case, I am simply doubling the dataset we had previously, so just copy and append the same dataset:
```
cp data/data.xml /tmp/data.xml
cat /tmp/data.xml >> data/data.xml
```

It's so similar to `git` that it almost feels natural! What would you do in Git? `git add`, right?
```
dvc add data/data.xml
```
Now, just in case, commit the changes in git, and then push the changes in your DVC remote storage

```
git add data/data.xml.dvc
git commit data/data.xml.dvc -m "Dataset updates"
```

And now simply push the changes:
```
dvc push
```

If you want to go back to a previous version of your data, or data stored in a different branch, you can git checkout to the commit or branch with the corresponsing version, and then run dvc checkout.

For example, if you want to revert to the original version of your dataset (before that, check that your current data.xml file has 50000 samples):
```
git checkout HEAD~1 data/data.xml.dvc
dvc checkout
```
If everything worked as expected, you should receive the following message: 
```
$ dvc checkout
M       data\data.xml
```

> ## Use `git checkout` followed by `dvc checkout` to get a previous version of your data

Also, notice that right now you have 25000 samples (which is the original length of the dataset)

## Is DVC only for data?

Absolutely not! Think about DVC as a Git for both data and models. You will train models, and then you can track them using the same commands you saw above. That way, when you run a `pull` or a `checkout` you will retrieve the data and the corresponding model.

Take a minute to reflect on what is going on. We are tracking files using Git, AND inside the repository we are adding large files (they could be data files, or model files). Then we add those large files to the DVC tracker, and at that moment, DVC will include a `.gitignore` so the original large file is not tracked by Git anymore, instead, Git will track the pointer to the original data, which is the file `data.xml.dvc`. 

One thing we haven't done yet is pushing the changes to a remote repository.

## Try it out

1. Create an __empty__ GitHub repository. 
2. Add remote to your dvc folder
3. Push the changes
4. Go to your repo and observe what has been uploaded

If you followed all instructions, you will see that, effectively, we haven't uploaded any large files!

# Retrieving data

Now that you know how to track files and to push them to remote storage services using DVC, how can you share your progress with your teammates? They can make a `dvc pull` to retrieve the files that has been tracked by DVC, but they have to get the tracking file first. Thus, DVC provides different commands for downloading data to your local machine very easily.

First, it would be useful to know what files in your project are being tracked. If you followed all the steps, you should have a GitHub repo with the file pointing towards our data stored in the S3 bucket.

<div style="text-align:center"><img src="images/DVC_github.png" width=600/></div>

I can check what files are tracked by DVC in my repo by running:
```
dvc list https://github.com/IvanYingX/dvc_test.git
```
Or, if you want to see all the content in the subdirectories:
```
dvc list -R https://github.com/IvanYingX/dvc_test.git
```
This is what I obtained, in your case, it should be something very similar (make sure you are pointing to the right repo, and not mine!)

<div style="text-align:center"><img src="images/DVC_List.png" width=350/></div>

Anyone could see the contents of the repo as long as they have the URL for the repo. Observe that `data.xml` is shown even though it doesn't appear in the repository. 

Now, if anyone who doesn't have this project on their local machine wants to download `data.xml`, they can run `dvc get`, which would be the equivalent of `wget` but for files tracked by DVC:
```
dvc get https://github.com/IvanYingX/dvc_test.git data.xml
```
_In this case, if you want to test it, go to a new directory (Desktop for example, and run the command above)_

In my case, I downloaded the whole folder:
```
cd ~/Desktop && dvc get https://github.com/IvanYingX/dvc_test.git data
```
or alternatively:
```
dvc get -o ~/Desktop https://github.com/IvanYingX/dvc_test.git data
```
which created a new folder in my desktop called `data`. If you look inside it, there is no `data.xml.dvc` file. That is because `dvc get` doesn't create a tracking system, but simply downloads the tracked files.

> ## `dvc get` will simply download the data, but it won't generate a DVC tracking file

## Try it out

The DVC team has many examples on their website and their repos. One is a set of pictures of cats and dogs that you can eventually use for training a model

1. Use dvc list to check what is being tracked in the following repo: https://github.com/iterative/dataset-registry
2. Use it once again to check what is inside the directory `use-cases`
3. Run a dvc get on the use-cases/cats-dogs directory (It will take some minutes, so let's take a break and we will come back in 10 minutes)
4. Observe what you have downloaded

If you wanted to keep track of these files, you would have to use `dvc add`. DVC has a shortcut for files that have been downloaded using `dvc get` which is `dvc import`.

## Try it out

1. Create a new folder
2. Initialize a repo `git init`
3. Initialize a DVC repo `dvc init`
4. Run `dvc import` with the `data.xml` you pushed previously 
5. Observe that now, you have many more files compared to what you obtained using `dvc get`

# DVC Pipelines

Keeping track and retrieving large files and models looks fine, but we want to train the model without all the hussle of pulling, training, pushing, tracking... DVC not only allows you to track data, but it also allows you to create pipelines to reproduce experiments.

Once again, let's use an example provided by the DVC team. Create a new folder, and run the following commands on your command line:
```
wget https://code.dvc.org/get-started/code.zip
unzip code.zip
rm -f code.zip
```
If for any reason you don't have `wget` installed, go to the following page: [https://www.jcchouinard.com/wget/](https://www.jcchouinard.com/wget/)

Your new folder should look like this:
<div style="text-align:center"><img src="images/DVC_Pipe.png" width=300/></div>

Take a couple of minutes to observe what is inside each file. Observe that there is a `requirements.txt` file. It might be worth it using a virtual environment for running the following codes.

Make sure to initialize the source folder repo `git init`, and add and commit everything. Don't worry about what is inside each file, just bear in mind that `featurization.py` will transform some raw data into features. We will also need the data we were using previously with the `data.xml` file. So, in the same directory, do a `dvc get`. In my case, it will be:
```
dvc get https://github.com/IvanYingX/dvc_test.git data
```
And as before, initialize DVC in your repo, add the large files you want to track, and make the corresponding commits:
```
dvc init
dvc add data/data.xml
dvc remote add -d storage s3://dvc-aicore/AiCore_DVC/
git add 'data/data.xml.dvc' 'data/.gitignore' '.dvc/config'
git commit -m 'Add DVC tracker and storage'
dvc push
```

Let's start creating the pipeline! Remember that pipelines can be treated as workflows with many tasks in line (so a linear DAG essentially). You can create a pipeline creating a yaml file and name it `dvc.yaml`, where each step of the pipeline will be defined.

Luckily, DVC has a command `dvc run` that helps you create these DAGs, and store it in a .yaml file

Each step is called stage, and everytime we execute `dvc run` we are creating a new stage. Let's create the first stage:
```
dvc run -n prepare \
        -p prepare.seed,prepare.split \
        -d src/prepare.py -d data/data.xml \
        -o data/prepared \
        python src/prepare.py data/data.xml
```

Click on each line to see the details

<details>
  <summary> -n prepare </summary>
  This will give the stage a name ('prepare' in this case)
</details>
<details>
  <summary> -p prepare.seed,prepare.split </summary>
  The p stands for parameters. It will look for a file named params (In this case 'params.yaml'). Them, it will use the prepare.seed and prepare.split parameters from that file
</details>
<details>
  <summary> -d src/prepare.py -d data/data.xml </summary>
  The d stands for dependencies. It tells the pipeline what files are necessary in this stage. If they are not in your repo, your DAG will ask you to include these files
</details>
<details>
  <summary> -o data/prepared </summary>
  The output created after running the files. 
</details>
<details>
  <summary> python src/prepare.py data/data.xml </summary>
  This is the actual command that is going to be ran. 
</details>

Once you run the code, you will see two new files, and that `data/.gitignore` is updated. `data/.gitignore` is updated so that you dont track the data split into train and test. The other two files are representations of the DAG, which, in this case, it only contains a single stage.

<div style="text-align:center"><img src="images/DVC_Stage.png" width=400/></div>

Once you have a pipeline, you can start making experiments by running `dvc repro`. But before that, let's add a stage to the pipeline, otherwise we would only have one step! To visualize the stages in your DAGs, you can run `dvc dag`, which will allow you to nicely see the pipeline:

<div style="text-align:center"><img src="images/DVC_Dag.png" width=200/></div>

Let's add a couple of steps to this DAG. The next step will take the prepared data and make features. These features will be reflected in `data/features`, and the stage will depend on featurization.py and the prepared data

```
dvc run -n featurize \
        -p featurize.max_features,featurize.ngrams \
        -d src/featurization.py -d data/prepared \
        -o data/features \
        python src/featurization.py data/prepared data/features
```

Upon running this code, your `dvc.yaml` file will be updated:

<div style="text-align:center"><img src="images/DVC_Stage_2.png" width=400/></div>

Also, notice that there are new pickle files in `data/features`

The last step will correspond to the training step:
```
dvc run -n train \
        -p train.seed,train.n_est,train.min_split \
        -d src/train.py -d data/features \
        -o model.pkl \
        python src/train.py data/features model.pkl
```

After running all these commands, some files should be commited using git
```
git add dvc.yaml dvc.lock .gitignore data/.gitignore
```

If you run `dvc dag`, the new DAG should look like this:

<div style="text-align:center"><img src="images/DVC_Dag_2.png" width=200/></div>

As you were creating the stages, all steps were executed. You can avoid running them by adding `--no-exec` when creating them.

However, the whole point of this is to automate the steps, so next time you don't have to specify dependencies between tasks. You can run all the defined steps simply executing `dvc repro` to reproduce complete or partial pipelines by executing their stages. If you run it now, DVC will detect that there are no changes, so let's change some parameters and see what happens.

Some things you could change in this pipeline are the train/test split size, the data itself, or the number of maximum features. Let's change the train/test splitting size. In `params.yaml` set a different value for split (make it sensible!), and execute `dvc repro`

You should see all steps running sequentially because the first stage ('prepare') depends on that parameter. However, if you change 'n_est' in `params.yaml`, and rerun `dvc repro` let's see what happens:

<div style="text-align:center"><img src="images/DVC_Repro.png" width=400/></div>

DVC detects that the dependencies of some steps haven't changed at all, so they can simply be skipped, and thus save memory

# Tracking Metrics

Creating DAGs is a nice way to automate a series of steps that will run sequentially. But we still haven't added anything for keeping track of how well each experiment is performing. We'll save that for later!